## Mendeley datasets download via API

Get your Mendeley API token by registering an app here:

https://dev.mendeley.com/myapps.html

In [22]:
import requests
from requests.auth import HTTPBasicAuth
import os
import pandas as pd
import textwrap
from IPython.display import clear_output

ROOT = ".." # Adjust to repository root

from dotenv import load_dotenv
DOTENV_PATH = os.path.join(ROOT,"../../apis/.env") # Adjust to .env file location

if load_dotenv(DOTENV_PATH):
    abs_path = os.path.abspath(DOTENV_PATH)
    drive, rel = os.path.splitdrive(abs_path)
    parts = rel.strip(os.sep).split(os.sep)
    masked = parts.copy()
    masked_parts = 1  # adjust to mask parts of your path
    max_maskable = max(0, len(parts) - 2)
    for i in range(min(masked_parts, max_maskable)):
        idx = len(parts) - 3 - i
        masked[idx] = "***"
    prefix = f"{drive}{os.sep}" if drive else (os.sep if abs_path.startswith(os.sep) else "")
    print(f"Loaded .env from {prefix}{os.sep.join(masked)}")
else:
    print("Failed to load .env file.")

CLIENT_ID = os.getenv("MENDELEY_CLIENT_ID")
CLIENT_SECRET = os.getenv("MENDELEY_CLIENT_SECRET")

def mask_token(token, unmasked_chars=3):
    return token[:unmasked_chars] + '*' * (len(token) - unmasked_chars*2) + token[-unmasked_chars:]
print(f"CLIENT_SECRET: {mask_token(CLIENT_SECRET)}")

Loaded .env from c:\Users\david\***\apis\.env
CLIENT_SECRET: hF8**********r63


## Worldwide Electricity Load Dataset

## Global Day-Ahead Electricity Price Dataset

Ullah, Md Habib; Reza, Sayed Mohsin; Gundapaneni, Lasya Madhuri; Balachander, Pranav; Babaiahgari, Bhanu; Khan, Abdullah Al Ahad (2025), “Global Day-Ahead Electricity Price Dataset”, Mendeley Data, V3, doi: 10.17632/s54n4tyyz4.3
https://data.mendeley.com/datasets/s54n4tyyz4/3

In [25]:
DATASET_ID = "ybggkc58fz"
DATASET_VERSION = "1"

TOKEN_URL = "https://api.mendeley.com/oauth/token"
DATASET_URL = f"https://api.mendeley.com/datasets/{DATASET_ID}"

# obtain access token
token_resp = requests.post(
    TOKEN_URL,
    data={"grant_type": "client_credentials", "scope": "all"},
    auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
)
token_resp.raise_for_status()
access_token = token_resp.json()["access_token"]

# access dataset metadata
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/vnd.mendeley-public-dataset.1+json",
}
params = {"version": DATASET_VERSION}

dataset_resp = requests.get(DATASET_URL, headers=headers, params=params)
dataset_resp.raise_for_status()

print(dataset_resp.json()["name"], '\n')
print(textwrap.fill(dataset_resp.json()["description"], width=120))

data = dataset_resp.json()

print("\nNumber_of_files:", len(data.get("files", [])), "\n")

files = data.get("files", [])
preview_names = ", ".join(file["filename"] for file in files[:26]) + "..."
print(textwrap.fill(preview_names, width=120))

Worldwide Electricity Load Dataset  

This dataset provides a comprehensive, global-scale collection of time-series electricity load data aggregated from
major power markets across Asia, Europe, North America, and Oceania. It captures system-level demand profiles at
consistent temporal resolution, enabling detailed analysis of consumption patterns, peak-demand behavior, and seasonal
variability across regions. The dataset is structured in a standardized and harmonized format to support load
forecasting, cross-market comparative studies, and data-driven research in power system planning, operation, and energy
market analytics.

Number_of_files: 489 

Additional_Information.csv, Australia_AEMO_2006.csv, Australia_AEMO_2007.csv, Australia_AEMO_2008.csv,
Australia_AEMO_2009.csv, Australia_AEMO_2010.csv, Australia_AEMO_2011.csv, Australia_AEMO_2012.csv,
Australia_AEMO_2013.csv, Australia_AEMO_2014.csv, Australia_AEMO_2015.csv, Australia_AEMO_2016.csv,
Australia_AEMO_2017.csv, Australia_AEMO

Downloading the full dataset via Mendeley API:

In [26]:
DATASET_DIR = os.path.join(ROOT, "datasets", "daep_api")
PKL_DIR = os.path.join(ROOT, "pkl_api")
LOG_PATH = os.path.join(ROOT, "datasets", "daep_api", "0_download_errors.log")

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)

In [27]:
results = []

files = data.get("files", [])

for f in files:
    file_name = f.get("filename")
    content = f.get("content_details", {})

    if not file_name or content.get("content_type") != "text/csv":
        continue

    file_path = os.path.join(DATASET_DIR, file_name)
    pickle_file = os.path.join(PKL_DIR, file_name.replace(".csv", ".pkl"))

    try:
        # skip download if CSV already exists
        if not os.path.exists(file_path):
            download_url = content.get("download_url")
            if not download_url:
                continue

            r = requests.get(download_url, stream=True, timeout=60)
            r.raise_for_status()

            with open(file_path, "wb") as fh:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        fh.write(chunk)

        df = pd.read_csv(file_path)
        record_count = len(df)

        results.append({"csv_name": file_name, "record_count": record_count})
        print(f"Processed {file_name} with {record_count} records.")
        clear_output(wait=True)

        if "Timestamp" in df.columns:
            df["Timestamp"] = pd.to_datetime(df["Timestamp"])

        os.makedirs(os.path.dirname(pickle_file), exist_ok=True)
        df.to_pickle(pickle_file)

    except Exception as e:
        with open(LOG_PATH, "a", encoding="utf-8") as log:
            log.write(f"{file_name} | {type(e).__name__} | {e}\n")
        continue

csvs_df = pd.DataFrame(results)
csvs_df["pkl_name"] = csvs_df["csv_name"].str.replace(".csv", ".pkl")

display(csvs_df)

list_file_name = "0_daep_files_list.pkl"
csvs_df.to_pickle(os.path.join(PKL_DIR, list_file_name))
print(f"Saved file list to {os.path.join(PKL_DIR, list_file_name)}")

,csv_name,record_count,pkl_name
0,Additional_Information.csv,42,Additional_Information.pkl
1,Australia_AEMO_2006.csv,17519,Australia_AEMO_2006.pkl
2,Australia_AEMO_2007.csv,17520,Australia_AEMO_2007.pkl
3,Australia_AEMO_2008.csv,17568,Australia_AEMO_2008.pkl
4,Australia_AEMO_2009.csv,17520,Australia_AEMO_2009.pkl
...,...,...,...
484,USA_SPP_2021.csv,8758,USA_SPP_2021.pkl
485,USA_SPP_2022.csv,8749,USA_SPP_2022.pkl
486,USA_SPP_2023.csv,8757,USA_SPP_2023.pkl
487,USA_SPP_2024.csv,8782,USA_SPP_2024.pkl


Saved file list to ..\pkl_api\0_daep_files_list.pkl
